# E.16. 프로젝트 - Movielens 영화 SBR
##  Movielens 1M Dataset을 기반으로, Session based Recommendation 시스템 제작

### 평가기준
1. Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다.<br>
- 데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다.<br>
=> 
2. RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다. <br>
- 적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다.<br>
=>
3. 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.<br>
- 3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다.<br>
=>

In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime as dt
import datetime

import os
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

print(pd.__version__)
print(tf.__version__)

1.3.3
2.6.0


In [9]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], 
                       dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data['Time2'] = data['Time'].apply(lambda t: dt.fromtimestamp(t)) # Timestamp 타입을 datetime으로 변경 함.
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data



,UserId,ItemId,Rating,Time,Time2
31,1,3186,4,978300019,2000-12-31 22:00:19
22,1,1270,5,978300055,2000-12-31 22:00:55
27,1,1721,4,978300055,2000-12-31 22:00:55
37,1,1022,5,978300055,2000-12-31 22:00:55
24,1,2340,3,978300103,2000-12-31 22:01:43
...,...,...,...,...,...
1000019,6040,2917,4,997454429,2001-08-10 14:40:29
999988,6040,1921,4,997454464,2001-08-10 14:41:04
1000172,6040,1784,3,997454464,2001-08-10 14:41:04
1000167,6040,161,3,997454486,2001-08-10 14:41:26


## Step 1. 데이터의 전처리

In [10]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

In [6]:
oldest, latest = data['Time2'].min(), data['Time2'].max()
print(oldest) 
print(latest)

# 대략 3년치의 자료를 가지고 있다.

2000-04-25 23:05:32
2003-02-28 17:49:50


In [12]:
month_ago = latest - datetime.timedelta(30*2)  # 최종 날짜로부터 30 * 2 일 이전 날짜를 구한다.  
data = data[data['Time2'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data

,UserId,ItemId,Rating,Time,Time2
8704,59,2997,4,1041962568,2003-01-07 18:02:48
8721,59,2147,2,1041962639,2003-01-07 18:03:59
8781,59,2369,4,1041962664,2003-01-07 18:04:24
8797,59,3408,4,1041962692,2003-01-07 18:04:52
8808,59,2712,3,1041962692,2003-01-07 18:04:52
...,...,...,...,...,...
984731,5950,3948,4,1046369637,2003-02-27 18:13:57
984682,5950,3578,4,1046369670,2003-02-27 18:14:30
984475,5950,3793,3,1046369710,2003-02-27 18:15:10
984660,5950,3555,2,1046369737,2003-02-27 18:15:37


In [13]:
oldest, latest = data['Time2'].min(), data['Time2'].max()
print(oldest) 
print(latest)

2002-12-30 18:21:02
2003-02-28 17:49:50
